In [ ]:
import polars as pl
import squarify
import rtsvg
rt = rtsvg.RACETrack()
edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.rt_layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df            = pl.DataFrame(_lu_)
df            = df.with_columns(pl.col('fm').cast(pl.String), pl.col('to').cast(pl.String))
g             = rt.createNetworkXGraph(df, [('fm','to')])
_nodes_       = list(set(df['fm']) | set(df['to']))
_node_colors_ = {}
for i in range(98): _node_colors_[_nodes_.pop(0)] = 'red'
for i in range(24): _node_colors_[_nodes_.pop(0)] = 'yellow'
for i in range(41): _node_colors_[_nodes_.pop(0)] = 'green'
for i in range(2):  _node_colors_[_nodes_.pop(0)] = 'black'

_tiles_ = []
_pos_   = rt.treeMapNodeColorPlacement(g, set(df['fm']) | set(df['to']), _node_colors_)
_tiles_.append(rt.link(df, [('fm','to')], pos=_pos_, node_color=_node_colors_, w=512, h=512))
_pos_   = rt.treeMapNodeColorPlacement(g, set(df['fm']) | set(df['to']), _node_colors_, collapse=True)
_tiles_.append(rt.link(df, [('fm','to')], pos=_pos_, node_color=_node_colors_, w=512, h=512))
_tiles_.append(rt.link(df, [('fm','to')], pos=_pos_, w=512, h=512))
rt.tile(_tiles_, spacer=10)